# Reverse Typing Game (Türkçe)


**Bu proje, kullanıcıya verilen kelimeleri TERS şekilde yazmasını isteyen
mini bir web oyunudur. Flask + HTML + JavaScript teknolojileri kullanılmıştır.**

 Özellikler:
- 3 Zorluk modu: Kolay, Orta, Zor
- 60 saniyelik süre limiti
- Her doğru cevapta puan kazanma
- Oyun sonunda toplam puan gösterilir




In [1]:
# Flask modülünü içe aktar

from flask import Flask, render_template_string
import random  # Rastgele kelime seçimi için


In [1]:
# Zorluk seviyelerine göre kelimeler
# Kolay: kısa kelimeler
# Orta: kısa cümleler
# Zor: uzun cümleler


EASY_WORDS = [
    "elma", "çay", "kitap", "masa", "kedi", "köpek", "araba",
    "kalem", "su", "ev", "okul", "deniz", "ışık", "ağaç", "renk"
]

MEDIUM_WORDS = [
    "iyi sabahlar", "mutlu hafta", "yeni gün", "veri analizi",
    "flask projesi", "bugün hava güzel", "kod yazmak güzel",
    "bilgisayar mühendisliği", "başarı azim ister"
]

HARD_WORDS = [
    "trabzonspor şampiyon olacak", "yapay zeka gelecektir",
    "hayal et inşa et başar", "zor zamanlar güç yaratır",
    "geleceği kodla", "bilgi güçtür her zaman",
    "hayat bir oyundur öğren", "sabır kazandırır"
]


In [3]:
# Flask uygulaması oluşturuluyor

from flask import Flask, render_template_string
app = Flask(__name__)

# HTML Şablonu
# Bootstrap 5.3 koyu tema
# 3 zorluk seçeneği (Kolay, Orta, Zor)
# Süre ve skor göstergeleri
# Ana menü ve oyun ekranı


TEMPLATE = """
<!doctype html>
<html lang="tr" data-bs-theme="dark">
<head>
  <meta charset="utf-8">
  <title>Ters Yazma Oyunu 🎯</title>
  <meta name="viewport" content="width=device-width,initial-scale=1">
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet">

  <style>
    /* 🎨 Koyu arayüz teması ve genel stil ayarları */
    body { background:#0b0e12; color:#e7eef8; }
    .card { background:#121722; border:1px solid #1f2a3a; }
    .mono { font-family: monospace; }
    .ok { color:#18c964; } /* Doğru cevap rengi */
    .bad { color:#ff6b6b; } /* Yanlış cevap rengi */
    .pill { border-radius:999px; padding:.25rem .6rem; border:1px solid #2a3b56; }
    .blink { animation:blink 1s linear infinite; }
    @keyframes blink { 0%{opacity:1} 50%{opacity:.25} 100%{opacity:1} }
  </style>
</head>

<body>
<div class="container py-5 text-center">
  <h1 class="mb-4 fw-bold">🎮 Ters Yazma Oyunu</h1>

  <!-- 🧭 ZORLUK SEÇİM MENÜSÜ -->
  <div id="menu">
    <h5 class="text-secondary mb-3">Zorluk Seç:</h5>
    <div class="d-flex justify-content-center gap-3 mb-4">
      <!-- Zorluk butonları -->
      <button class="btn btn-success btn-lg" onclick="selectDifficulty('easy')">Kolay 🟢</button>
      <button class="btn btn-warning btn-lg" onclick="selectDifficulty('medium')">Orta 🟡</button>
      <button class="btn btn-danger btn-lg" onclick="selectDifficulty('hard')">Zor 🔴</button>
    </div>
  </div>

  <!-- 🕹️ OYUN EKRANI -->
  <div id="game" style="display:none;">
    <!-- Süre ve puan göstergeleri -->
    <div class="d-flex justify-content-center gap-3 mb-4">
      <span class="pill">⏱️ Süre: <b id="time">60</b>s</span>
      <span class="pill">⭐ Puan: <b id="score">0</b></span>
    </div>

    <!-- Başla ve Ana Menü butonları -->
    <div class="d-flex justify-content-center gap-2 mb-4">
      <button id="startBtn" class="btn btn-lg btn-primary">Başla ▶️</button>
      <button id="backBtn" class="btn btn-outline-light">Ana Menüye Dön 🔁</button>
    </div>

    <!-- 🎯 Kelime Gösterimi ve Girdi Alanı -->
    <div class="card p-4 mx-auto" style="max-width:600px;">
      <div class="mb-2 text-secondary">Kelime:</div>
      <div id="target" class="mono h4 mb-3"></div>
      <input id="input" class="form-control form-control-lg mono text-center"
             placeholder="Tersini yaz ve Enter’a bas..." disabled autocomplete="off">
      <div id="feedback" class="mt-3 small"></div>
    </div>

    <!-- 🏁 Oyun Sonu Mesajı -->
    <div id="endMessage" class="mt-4 fw-bold h4 text-warning"></div>
  </div>

  <!-- 📄 Sayfa Alt Bilgisi -->
  <footer class="mt-5 text-secondary small">
   Gökdeniz Sağlam
  </footer>
</div>

<!-- JavaScript kısmı buraya eklenecek -->
"""



In [4]:
# JavaScript tarafı (oyun mantığı)

# Kullanıcıya verilen kelimeyi ters yazar
# Doğruysa puan artar
# Süre: 60 saniye (bitince otomatik durur)

JS_LOGIC = """
<script>
(function(){
  // 🔤 Python'dan gelen kelimeleri al
  const allWords = {
    easy: {{ easy|tojson }},
    medium: {{ medium|tojson }},
    hard: {{ hard|tojson }}
  };

  // 🧩 Oyun değişkenleri
  let words = [];          // aktif kelime listesi
  let current = "";         // şu anda ekranda olan kelime
  let score = 0;            // toplam puan
  let timeLeft = 60;        // süre (sabit 60 saniye)
  let timerId = null;       // zamanlayıcıyı tutar
  let playing = false;      // oyun açık mı kapalı mı?
  let difficulty = "";      // seçilen zorluk seviyesi
  let pointValue = 10;      // her doğru için puan (zorlukla değişir)

  // 📦 HTML öğelerini seç
  const elMenu = document.getElementById("menu");
  const elGame = document.getElementById("game");
  const elTarget = document.getElementById("target");
  const elInput = document.getElementById("input");
  const elScore = document.getElementById("score");
  const elTime = document.getElementById("time");
  const elFeedback = document.getElementById("feedback");
  const elStart = document.getElementById("startBtn");
  const elEnd = document.getElementById("endMessage");
  const elBack = document.getElementById("backBtn");

  // 🔄 Basit yardımcı fonksiyon: kelimeyi ters çevir
  const reverse = s => s.split("").reverse().join("");

  // 🟡 Zorluk seçimi
  window.selectDifficulty = function(diff){
    difficulty = diff;
    words = [...allWords[diff]]; // listeyi kopyala (orijinali bozulmasın)
    pointValue = diff === "easy" ? 10 : diff === "medium" ? 15 : 20;

    // Menüden oyun ekranına geç
    elMenu.style.display = "none";
    elGame.style.display = "block";

    // Oyunu sıfırla
    resetGame();
  }

  // ♻️ Oyun sıfırlama fonksiyonu
  function resetGame(){
    score = 0;
    timeLeft = 60;
    elScore.textContent = score;
    elTime.textContent = timeLeft;
    elTarget.textContent = "";
    elInput.value = "";
    elInput.disabled = true;
    elEnd.textContent = "";
    elFeedback.textContent = "";
    clearInterval(timerId); // varsa eski zamanlayıcıyı sıfırla
  }

  // 🎯 Yeni kelime getir
  function nextWord(){
    // Tüm kelimeler bittiyse, yeni listeyi tekrar yükle
    if(words.length === 0){
      words = [...allWords[difficulty]];
    }

    // Rastgele kelime seç
    const index = Math.floor(Math.random() * words.length);
    current = words[index];
    words.splice(index, 1); // aynı kelime tekrar çıkmasın
    elTarget.textContent = current;
    elInput.value = "";
    elInput.focus(); // kullanıcı hemen yazabilsin
  }

  // ⏳ Süre sayacı başlat
  function startTimer(){
    timerId = setInterval(()=>{
      timeLeft--;
      elTime.textContent = timeLeft;

      // Süre azaldığında yanıp sönme efekti
      if(timeLeft <= 10) elTime.classList.add("blink");

      // Süre bittiğinde oyunu durdur
      if(timeLeft <= 0) stopGame("⏰ Süre doldu!");
    },1000);
  }

  // ▶️ Oyun başlat
  function startGame(){
    resetGame();
    playing = true;
    elStart.disabled = true;
    elInput.disabled = false;
    nextWord();
    startTimer();
  }

  // 🛑 Oyun bitiş fonksiyonu
  function stopGame(msg){
    playing = false;
    clearInterval(timerId);
    elInput.disabled = true;
    elStart.disabled = false;
    elEnd.textContent = msg + " Toplam puanın: " + score;
  }

  // ⌨️ Enter tuşuna basılınca kontrol et
  elInput.addEventListener("keydown", e => {
    if(!playing) return;
    if(e.key === "Enter"){
      const typed = elInput.value.trim().toLowerCase(); // kullanıcının yazdığı
      const correct = reverse(current); // doğru cevap
      if(typed === correct){
        // ✅ Doğruysa puan ekle
        score += pointValue;
        elScore.textContent = score;
        elFeedback.innerHTML = "<span class='ok'>✅ Doğru! +" + pointValue + " puan</span>";
      } else {
        // ❌ Yanlışsa sadece uyarı
        elFeedback.innerHTML = "<span class='bad'>❌ Yanlış! Doğrusu: " + correct + "</span>";
      }
      setTimeout(()=> elFeedback.textContent = "", 700);
      nextWord(); // bir sonraki kelimeye geç
    }
  });

  // 🔙 Ana menüye dön
  elBack.addEventListener("click", ()=>{
    clearInterval(timerId);
    elGame.style.display = "none";
    elMenu.style.display = "block";
    resetGame();
  });

  // 🎬 Başla butonu
  elStart.addEventListener("click", startGame);
})();
</script>
"""


In [ ]:
# Flask route: Ana sayfa


@app.route("/")
def home():
    html = TEMPLATE + JS_LOGIC + "</body></html>"
    return render_template_string(
        html,
        easy=EASY_WORDS,
        medium=MEDIUM_WORDS,
        hard=HARD_WORDS
    )

# Sunucuyu başlat

if __name__ == "__main__":
    app.run(debug=False, use_reloader=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Oct/2025 14:00:46] "GET / HTTP/1.1" 200 -
